In [ ]:
import sys
sys.path.insert(0, '../')

import time
import numpy as np
import matplotlib.pyplot as plt
from rand_exp_tilted_stable import ExpTiltedStableDist

## Specify the range of parameters to run the samplers

In [ ]:
bridge_exponent = 1. / 16
divide_conquer_cost = 10 ** np.linspace(-1., 1., 101)

char_exponent = bridge_exponent / 2
tilt = divide_conquer_cost ** (1. / char_exponent)
# For Bayesian bridge, tilt parameter is given by beta / global_scale
tilt_power = tilt ** char_exponent

## Time the samplers at given parameter values

In [ ]:
tilted_stable = ExpTiltedStableDist(seed=0)

def time_method(char_exponent, tilt, method, n_rep=1000):
    start = time.time()
    tilted_stable.sample(
        char_exponent * np.ones(n_rep), tilt * np.ones(n_rep), 
        method=method
    );
    elapsed = time.time() - start
    return elapsed

In [ ]:
n_repetition = 1000

exec_time = {
    method: 
        np.array([
            time_method(char_exponent, tilt_i, method, n_repetition)
            for tilt_i in tilt
        ]) 
    for method in ['double-rejection', 'divide-conquer']
}

## Plot the result

In [ ]:
plt.figure(figsize=(7, 4.5))
plt.rcParams['font.size'] = 18

for method in ['double-rejection', 'divide-conquer']:
    plt.plot(tilt_power, exec_time[method] / n_repetition)
plt.xlabel('Cost of divide-conquer (= tilt ^ char-exponent)')
plt.ylabel('Sec. per sample')
plt.ticklabel_format(axis='y', scilimits=(0,0))
plt.ylim(bottom=0)

for side in ['top', 'right']:
    plt.gca().spines[side].set_visible(False)

## Run a basic check to confim the sampler chooses the faster method 
Call the method with randomly generated tilting parameters. If the sampler correctly chooses the faster method for each parameter setting, then it should run faster than using one fixed method for all the parameter settings.

In [ ]:
n_samples = 10 ** 3

double_rejection_cost = 2.
divide_conquer_cost \
    = double_rejection_cost * np.random.exponential(size=n_samples)
tilt = divide_conquer_cost ** (1. / char_exponent)

In [ ]:
tilted_stable.set_seed(0)
%timeit -n 100 tilted_stable.sample(char_exponent, tilt)

tilted_stable.set_seed(0)
%timeit -n 100 tilted_stable.sample(char_exponent, tilt, 'double-rejection')

tilted_stable.set_seed(0)
%timeit -n 100 tilted_stable.sample(char_exponent, tilt, 'divide-conquer')